In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import Compose
from albumentations import Compose, ShiftScaleRotate, ToFloat
from albumentations.pytorch import ToTensorV2
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

In [26]:
df1 = pd.read_csv('train.csv.zip')
Y = df1.label.values
df1.drop(columns = 'label', inplace = True)
X = df1.values.reshape(-1,28,28,1) 
train_x, val_x, train_y, val_y = train_test_split(X,Y, test_size =0.2)

In [27]:
train_transforms = Compose([ToFloat(max_value = 255),ShiftScaleRotate(), ToTensorV2()])
val_transforms = Compose([ToFloat(max_value = 255), ToTensorV2()])

In [28]:
class dataloading(Dataset):
    def __init__(self, X,y = None, is_test = False, transforms = None ):
        self.X = X
        self.y = y
        self.is_test = is_test
        self.transforms = transforms
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        image = self.X[index]
        if self.transforms:
            image = self.transforms(image = image)['image']
        if self.is_test:
            return image
        else :
            return image,self.y[index]

In [29]:
class CNN(nn.Module) :
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 16, kernel_size = 3)
        self.batch1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels = 16,out_channels = 32,kernel_size = 3)
        self.batch2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(in_channels = 32,out_channels = 32,kernel_size = 5)
        self.batch3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32*8*8,100)
        self.batchfc1 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100,10)
        self.dropout = nn.Dropout2d(0.7)
        
    def forward(self,x):
        x = F.leaky_relu(self.conv1(x))
        x = self.batch1(x)
        x = F.leaky_relu(self.conv2(x))
        x = self.batch2(x)
        x = self.pool2(x)
        x = F.leaky_relu(self.conv3(x))
        x = self.batch3(x)
        x = x.view(x.size(0),-1)

        x = F.leaky_relu(self.fc1(x))
        x = self.batchfc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [30]:
def train_one_epoch(model,train_loader, optimizer, criterion):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader :
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss/len(train_loader.dataset)
    return train_loss

In [31]:
def validate(model,val_loader,criterion):
    model.eval()
    correct = 0
    running_val_loss = 0
    with torch.no_grad() :
        for inputs, labels in val_loader :
            outputs = model(inputs)
            loss= criterion(outputs,labels)
            pred = outputs.argmax(dim = 1)
            correct+= pred.eq(labels).sum().item()
            running_val_loss += loss.item()
    acc = correct/len(val_loader.dataset)
    val_loss = running_val_loss/len(val_loader.dataset)
    return acc, val_loss

In [32]:
def fit(model, train_loader, val_loader,learning_rate, num_epochs):
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = ExponentialLR(optimizer, gamma = 0.99)
    for epoch in range(1,num_epochs+1):
        train_loss = train_one_epoch(model, train_loader,optimizer, criterion)
        acc, val_loss = validate(model, val_loader,criterion)
        scheduler.step()
        print(f'Epoch No. = {epoch} \n Train loss = {train_loss} \n Val accuracy = {acc}' )

In [ ]:
train_batch_size = 64
val_batch_size = 256
learning_rate = 0.0001
num_epochs = 50

train_X = dataloading(train_x, train_y, is_test = False, transforms = train_transforms)
val_X = dataloading(val_x, val_y, is_test = False, transforms = val_transforms)

train_loader = DataLoader(train_X, batch_size = train_batch_size, drop_last = True,shuffle = True)
val_loader = DataLoader(val_X, batch_size = val_batch_size, shuffle = False)

model = CNN()
fit(model, train_loader, val_loader,learning_rate, num_epochs)

Epoch No. = 1 
 Train loss = 0.014894557040007342 
 Val accuracy = 0.959047619047619
Epoch No. = 2 
 Train loss = 0.00647108912113167 
 Val accuracy = 0.9742857142857143
Epoch No. = 3 
 Train loss = 0.004462609902423407 
 Val accuracy = 0.9791666666666666


In [ ]:
id = np.arange(1,28001,1)
df = pd.DataFrame(id, columns = ['ImageId'])
test = pd.read_csv('test.csv.zip')
test = test.values.reshape(-1,28,28,1)
test_transforms = Compose([ToFloat(max_value = 255), ToTensorV2()])
test_X = dataloading(test, is_test = True, transforms = val_transforms)
test_loader = DataLoader(test_X, batch_size = 28000, shuffle = False)
def predict(model, test_loader) :
    model.eval()
    with torch.no_grad() :
        for i, inputs in enumerate(test_loader):
            outputs = model(inputs)
            pred = outputs.argmax(dim =1).numpy()
    df2 = pd.DataFrame(pred, columns = ['Label'])
    output = pd.concat([df,df2], axis =1)
    output.to_csv(r'C:\Users\Pranav\Desktop\Projects\Digit Recognizer\out2', index = False)
    return output    

In [21]:
output = predict(model, test_loader,device)
output

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
